In [1]:
import tensorflow as tf 
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

c:\Users\saura\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Load trained load model
model = load_model("model.h5")

# Load encoder and scaler
with open("onehot_encoder_geo.pkl","rb") as file:
    label_encoder_geo = pickle.load(file)

with open("label_encoder_gender.pkl","rb")as file:
    label_encoder_gender = pickle.load(file)

with open("scaler.pkl","rb")as file:
    scaler = pickle.load(file)


In [3]:
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000

}

In [4]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded

c:\Users\saura\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[1., 0., 0.]])

In [5]:
geo_encoded_df = pd.DataFrame(geo_encoded,columns = label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
input_df = pd.DataFrame([input_data])

In [7]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [8]:
# Change the male to 1 by using label_encoder_gender.transform.

input_df['Gender'] = label_encoder_gender.transform([input_df['Gender']])
input_df

c:\Users\saura\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [9]:
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [10]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [11]:
# Scale your data
input_scaled = scaler.transform(input_df)

In [12]:
input_scaled

array([[-0.5152381 ,  0.91301738,  0.10285036, -0.68699257, -0.26063048,
         0.79790076,  0.6493972 ,  0.96728484, -0.8682109 ,  0.99850112,
        -0.57812007, -0.57542576]])

In [13]:
model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


array([[0.03831393]], dtype=float32)

In [14]:
prediction_proba = model.predict(input_scaled)[0][0]
prediction_proba

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


0.03831393

In [15]:
if prediction_proba > 0.5:
    print("The customer is likely to churn")
else:
    print("The custome is not likely to churn.")

The custome is not likely to churn.


In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ H_1 (Dense)                     │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ H_2 (Dense)                     │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ OUTPUT (Dense)                  │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,947 (11.52 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

# This are the training weights of the models. 

In [17]:
model.layers[0].weights

[<Variable path=sequential/H_1/kernel, shape=(12, 64), dtype=float32, value=[[ 1.78578183e-01  8.49694684e-02 -2.16794446e-01  3.90302017e-02
    1.34743720e-01  1.73019871e-01 -7.03480318e-02 -8.98421630e-02
   -3.02507263e-02  9.18923840e-02 -4.98765595e-02  5.35553806e-02
   -3.00938606e-01  1.48416802e-01  1.78634282e-02 -2.60350648e-02
   -2.99887389e-01  1.32333443e-01  2.15221643e-01 -1.92775771e-01
    8.25578570e-02 -1.87233210e-01  1.62920833e-01 -7.29773119e-02
    1.53747931e-01 -1.98680669e-01 -2.02422783e-01  1.84923977e-01
   -1.11656236e-02  2.10056230e-02  9.89659037e-03  8.62173140e-02
   -7.64741302e-02 -1.62313610e-01  2.08185911e-01  5.61998878e-03
   -9.12091210e-02  6.52125664e-03 -1.79878518e-01  1.19514875e-01
    1.19522497e-01 -7.65471440e-03  7.49759823e-02  1.79743562e-02
   -1.51977003e-01  1.24862961e-01  4.55295332e-02 -1.41623870e-01
    8.85466784e-02 -1.50423616e-01 -2.44620070e-02  6.68595508e-02
    3.87993634e-01 -1.26227364e-01  1.85932100e-01  1.

In [18]:
model.layers[0].weights[0].value[0]

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([ 0.17857818,  0.08496947, -0.21679445,  0.0390302 ,  0.13474372,
        0.17301987, -0.07034803, -0.08984216, -0.03025073,  0.09189238,
       -0.04987656,  0.05355538, -0.3009386 ,  0.1484168 ,  0.01786343,
       -0.02603506, -0.2998874 ,  0.13233344,  0.21522164, -0.19277577,
        0.08255786, -0.18723321,  0.16292083, -0.07297731,  0.15374793,
       -0.19868067, -0.20242278,  0.18492398, -0.01116562,  0.02100562,
        0.00989659,  0.08621731, -0.07647413, -0.16231361,  0.20818591,
        0.00561999, -0.09120912,  0.00652126, -0.17987852,  0.11951488,
        0.1195225 , -0.00765471,  0.07497598,  0.01797436, -0.151977  ,
        0.12486296,  0.04552953, -0.14162387,  0.08854668, -0.15042362,
       -0.02446201,  0.06685955,  0.38799363, -0.12622736,  0.1859321 ,
        0.17114094,  0.3040476 ,  0.03027617, -0.10750931,  0.05648769,
       -0.06873745, -0.07413608, -0.26768985,  0.00983334], dtype=float32)>

In [19]:
model.layers[0].weights[1].value.shape

TensorShape([64])